In [9]:
import pandas as pd
from sklearn import metrics,cluster

#### Reading the feature and calculating the pairwise distance between all the points

df = pd.read_csv('gh_lyso_PAAC.txt', sep='\t')# input file which contains features, header=None if no column title
x=df.iloc[:,1:].values
distance=metrics.pairwise.pairwise_distances(x)

#### Clustering

kmeans = cluster.KMeans(n_clusters=3,random_state=42).fit(x) # Number of clusters
labels = kmeans.labels_
centroid=kmeans.cluster_centers_
cent_label=kmeans.predict(centroid)


#### collecting intra and inter distance for clusters

class clust:
    def __init__(self,ref,comp):
        self.ref=ref
        self.comp=comp
        if self.ref==self.comp:
            print(f'The intra cluster distance within cluster {self.ref}')
            self.c=1
        else:
            print(f'The inter cluster distance between cluster {self.ref} and {self.comp}')
            self.c=0
        
    def all_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            for y1 in range((x1+1)*self.c,len(labels)):
                if labels[x1]==self.ref and labels[y1]==self.comp:
#                     print(f'{x1}..{y1}',distance[x1][y1])
                    al_data.append(distance[x1][y1])
        return al_data
                          
# a=clust(0,1).all_dist()
# b=clust(0,0).all_dist()

In [7]:
labels

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0])

In [8]:
centroid

array([[0.03153644, 0.00415653, 0.01368392, 0.00465001, 0.01252685,
        0.03202206, 0.00610189, 0.01199674, 0.00975414, 0.01648111,
        0.00470907, 0.01246332, 0.01031211, 0.00821828, 0.00679   ,
        0.02834656, 0.02097201, 0.0118598 , 0.00854663, 0.0160218 ,
        0.02475574, 0.02526016, 0.0244899 , 0.02387724, 0.02401027,
        0.02465904, 0.023474  , 0.02407751, 0.02485778, 0.02451014,
        0.02380502, 0.02462666, 0.02527053, 0.02422633, 0.02412836,
        0.02433256, 0.02304753, 0.02341199, 0.02346828, 0.02469479,
        0.02376041, 0.02380391, 0.02428294, 0.02464732, 0.02449224,
        0.02381464, 0.02482714, 0.025057  , 0.02490207, 0.02427924],
       [0.02099295, 0.01397598, 0.01367832, 0.00731533, 0.00590227,
        0.01858204, 0.00422606, 0.01109787, 0.01361521, 0.01966675,
        0.00383651, 0.01697063, 0.00437184, 0.00928663, 0.01553698,
        0.01563089, 0.01117948, 0.01592416, 0.00837854, 0.00920128,
        0.02592102, 0.02669973, 0.02349844, 0.0

In [10]:
cent_label

array([0, 1, 2])